## Undersampling y separación en train y validación

En este notebook, nos enfocamos en técnicas esenciales de preprocesamiento y sub-muestreo para optimizar conjuntos de datos destinados a modelos de machine learning. Comenzamos con la carga y preparación de datos de entrenamiento para escenarios destinados tanto a modelos de árboles de decisión y emsembled o modelos de regresión e implementamos estrategias de sub-muestreo para abordar desequilibrios en la distribución de las clases. A través de funciones personalizadas, equilibramos eficazmente los conjuntos de datos y preparamos subconjuntos de entrenamiento y validación. Este enfoque no solo mejora la calidad del entrenamiento de los modelos sino que también es crucial para obtener predicciones más precisas y fiables.

In [1]:
import pickle
from sklearn.model_selection import train_test_split
import pandas as pd
from imblearn.under_sampling import RandomUnderSampler
from imblearn.under_sampling import RandomUnderSampler
import numpy as np

Cargamos los datos de entrenamiento, tanto los datos destinados a modelos de regresión, NaiveBayes y SVC como los destinados a modelos de árboles y emsembled.
En unos datos, tal y como explicamos anteriormente se eliminaron las variables originales tras la creción de nuevas variables mientras que en otros datos se han mantenido.

In [2]:
import pickle

with open('../data/X_train.pkl', 'rb') as f:
    X_train = pickle.load(f)

with open('../data/y_train.pkl', 'rb') as f:
    y_train = pickle.load(f)

with open('../data/X_train_nb_lr_svc.pkl', 'rb') as f:
    X_train_nb_lr_svc= pickle.load(f)

with open('../data/y_train_nb_lr_svc.pkl', 'rb') as f:
    y_train_nb_lr_svc = pickle.load(f)


Definimos funciones que más adelante en el notebook nos serán útiles.

In [3]:
def apply_undersampling(X, y, ratio=9, random_state=42):
    """
    Función para aplicar sub-muestreo a los datos.

    Parámetros:
    - X: Características (features)
    - y: Etiquetas (labels)
    - ratio: Ratio deseado entre las clases 0 y 1 (n_zeros/n_ones)
    - random_state: Estado aleatorio para el sub-muestreo

    Retorna:
    - X_undersampled: X después del sub-muestreo
    - y_undersampled: y después del sub-muestreo
    """
    # Contar el número de ocurrencias de cada clase
    counts = np.bincount(y)

    # Calcular el número de muestras para cada clase para lograr la proporción deseada
    n_ones = counts[1]
    n_zeros = int(n_ones * ratio)
    sampling_strategy = {0: n_zeros, 1: n_ones}

    rus = RandomUnderSampler(sampling_strategy=sampling_strategy, random_state=random_state)
    X_undersampled, y_undersampled = rus.fit_resample(X, y)

    return X_undersampled, y_undersampled


def train_val_split_undersample(X, y, test_size=0.2, undersample_ratio=90, random_state_split=12345, random_state_sampling=42):
    """
    Divide los datos en conjuntos de entrenamiento y validación y luego aplica sub-muestreo al conjunto de entrenamiento.
    """
    # Dividir los datos en conjuntos de entrenamiento y validación
    X_train, X_val, y_train, y_val = train_test_split(X, y, stratify=y, test_size=test_size, random_state=random_state_split)

    # Contar el número de ocurrencias de cada clase en el conjunto de entrenamiento
    counts = np.bincount(y_train)

    # Calcular el número de muestras para lograr la proporción deseada
    n_ones = counts[1]
    n_zeros = int(n_ones * (undersample_ratio / (100 - undersample_ratio)))
    sampling_strategy = {0: n_zeros, 1: n_ones}

    # Aplicar sub-muestreo al conjunto de entrenamiento
    rus = RandomUnderSampler(sampling_strategy=sampling_strategy, random_state=random_state_sampling)
    X_train_val, y_train_val = rus.fit_resample(X_train, y_train)

    return X_train_val, y_train_val, X_val, y_val



Utilizamos la primera función llamada apply_undersampling. 

La función apply_undersampling está diseñada para equilibrar un conjunto de datos que tiene un desequilibrio en la distribución de sus clases. Se centra en el sub-muestreo de la clase mayoritaria para alcanzar una proporción específica en relación con la clase minoritaria.

La función primero cuenta la cantidad de muestras en cada clase.

Luego, calcula cuántas muestras de la clase mayoritaria (clase 0 en este caso) deben mantenerse para alcanzar el ratio deseado con respecto a la clase minoritaria (clase 1).

Aplica sub-muestreo utilizando RandomUnderSampler de la biblioteca imblearn, reduciendo la clase mayoritaria hasta alcanzar la proporción especificada. 

Finalmente retorna el conjunto de datos (X_undersampled y y_undersampled) con la clase mayoritaria sub-muestreada, logrando así una distribución de clase más equilibrada.

En nuestro caso aplicamos sobre nuestros dos conjuntos de datasets y elegimos que el 90% de las muestras en el conjunto de datos equilibrado serán de la clase 0, mientras que el 10% serán de la clase 1.

Estos dataset resultantes serán los que se utilicen posteriormente en el modelo definitivo para entrenarlo y posteriormente predecir el test.

In [4]:
X_train_undersampled, y_train_undersampled = apply_undersampling(X_train, y_train)
X_train_nb_lr_svc_undersampled, y_train_nb_lr_svc_undersampled = apply_undersampling(X_train_nb_lr_svc, y_train_nb_lr_svc)

Vemos como ha quedado distribuido y_train_undersampled en la variable objetivo.

In [5]:
y_train_undersampled.value_counts()

fraud_bool
0    79407
1     8823
Name: count, dtype: int64

Posteriormente cogemos el train inicial y lo dividimos en train y validación y posteriormente le hacemos undersampling solamente al train.

Esto lo hacemos en los dos conjuntos de datos. 

Estos conjuntos de datos son los que se utilizarán para entrenar y evaluar los modelos que nos servirán para decidir el modelo más optimo.

Esto lo hacemos a través de la funcion train_val_split_undersample.

La función realiza lo siguiente:

Divide un conjunto de datos en partes de entrenamiento y validación, y luego equilibra las clases en el conjunto de entrenamiento mediante sub-muestreo.

La función asegura que el conjunto de entrenamiento tenga una proporción más equilibrada de clases, mientras que el conjunto de validación refleja la distribución original de clases del conjunto de datos completo. Este enfoque ayuda a mejorar la eficacia del entrenamiento del modelo de machine learning, permitiendo que aprenda de manera más efectiva de ambas clases.
En esta ocasión también el 90% de las muestras en el conjunto de datos de entranamiento serán de la clase 0, mientras que el 10% serán de la clase 1.





In [6]:
X_train_val, y_train_val, X_val, y_val = train_val_split_undersample(X_train, y_train)
X_train_nb_lr_svc_val, y_train_nb_lr_svc_val, X_val_nb_lr_svc, y_val_nb_lr_svc= train_val_split_undersample(X_train_nb_lr_svc, y_train_nb_lr_svc)

In [7]:
y_train_val.value_counts()

fraud_bool
0    63522
1     7058
Name: count, dtype: int64

Por último guardamos todos los dataframes en pickles

In [8]:
import pickle

with open('../data/X_train_undersampled.pickle', 'wb') as file:
    pickle.dump(X_train_undersampled, file)

with open('../data/y_train_undersampled.pickle', 'wb') as file:
    pickle.dump(y_train_undersampled, file)

with open('../data/X_train_nb_lr_svc_undersampled.pickle', 'wb') as file:
    pickle.dump(X_train_nb_lr_svc_undersampled, file)

with open('../data/y_train_nb_lr_svc_undersampled.pickle', 'wb') as file:
    pickle.dump(y_train_nb_lr_svc_undersampled, file)

with open('../data/X_train_val.pickle', 'wb') as file:
    pickle.dump(X_train_val, file)

with open('../data/y_train_val.pickle', 'wb') as file:
    pickle.dump(y_train_val, file)

with open('../data/X_val.pickle', 'wb') as file:
    pickle.dump(X_val, file)

with open('../data/y_val.pickle', 'wb') as file:
    pickle.dump(y_val, file)

with open('../data/X_train_nb_lr_svc_val.pickle', 'wb') as file:
    pickle.dump(X_train_nb_lr_svc_val, file)

with open('../data/y_train_nb_lr_svc_val.pickle', 'wb') as file:
    pickle.dump(y_train_nb_lr_svc_val, file)

with open('../data/X_val_nb_lr_svc.pickle', 'wb') as file:
    pickle.dump(X_val_nb_lr_svc, file)

with open('../data/y_val_nb_lr_svc.pickle', 'wb') as file:
    pickle.dump(y_val_nb_lr_svc, file)
